In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "X-Linked_Lymphoproliferative_Syndrome"
cohort = "GSE190042"

# Input paths
in_trait_dir = "../../input/GEO/X-Linked_Lymphoproliferative_Syndrome"
in_cohort_dir = "../../input/GEO/X-Linked_Lymphoproliferative_Syndrome/GSE190042"

# Output paths
out_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/GSE190042.csv"
out_gene_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/gene_data/GSE190042.csv"
out_clinical_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/clinical_data/GSE190042.csv"
json_path = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE190042_family.soft.gz', 'GSE190042_series_matrix.txt.gz']
Identified SOFT files: ['GSE190042_family.soft.gz']
Identified matrix files: ['GSE190042_series_matrix.txt.gz']



Background Information:
!Series_title	"Integration between MCL1 gene co-expression module and the Revised International Staging System enables precise prognostication and prediction of response to proteasome inhibitor-based therapy in individual multiple myeloma"
!Series_summary	"We recently identified a gene module of 87 genes co-expressed with MCL1 (MCL1-M), a critical regulator of plasma cell survival. MCL1-M captures both MM cell-intrinsically acting signals and the signals regulating the interaction between MM cells with bone marrow microenvironment. MM can be clustered into MCL1-M high and MCL1-M low subtypes. While the MCL1-M high MMs are enriched in a preplasmablast signature, the MCL1-M low MMs are enriched in B cell-specific genes. In multiple independent datasets, MCL1-M high MMs exhibited poorer prognosis compared to MCL1-M low MMs. Re-analysis of the phase III HOVON-65/GMMG-HD4 showed that only MCL1-M MMs, but not MCL1-M low MMs, benefited from bortezomib-based treatment.

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Yes, this dataset has gene expression data from Affymetrix PrimeView array 
# based on the background information
is_gene_available = True

# 2. Data Availability and Conversion

# 2.1 Trait Availability - For X-Linked Lymphoproliferative Syndrome
# This dataset appears to be about Multiple Myeloma (MM), not X-Linked Lymphoproliferative Syndrome
# Looking at the sample characteristics, row 4 contains different MM types
trait_row = None  # Not available for this specific trait

# 2.2 Age Availability
age_row = 2  # Row 2 contains age information

# 2.3 Gender Availability
gender_row = 1  # Row 1 contains gender information

# Data Type Conversion Functions
def convert_trait(value):
    # Not needed since trait data is not available
    return None

def convert_age(value):
    """Convert age values to continuous numeric format."""
    if not value or ":" not in value:
        return None
    
    # Extract the value after the colon
    try:
        age_str = value.split(":")[1].strip()
        age = int(age_str)
        return age
    except (IndexError, ValueError):
        return None

def convert_gender(value):
    """Convert gender values to binary format (0 for female, 1 for male)."""
    if not value or ":" not in value:
        return None
    
    # Extract the value after the colon
    gender_str = value.split(":")[1].strip().upper()
    
    if gender_str == 'F':
        return 0
    elif gender_str == 'M':
        return 1
    else:
        return None

# 3. Save metadata - initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Skip the clinical feature extraction since trait_row is None


False

### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')

Gene expression data shape: (49395, 93)


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, these appear to be probe IDs (like 11715100_at) from a microarray platform,
# not standard human gene symbols. Probe IDs need to be mapped to gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


Sample of gene expression data (first 5 rows, first 5 columns):
               GSM5712490  GSM5712491  GSM5712492  GSM5712493  GSM5712494
ID                                                                       
11715100_at      2.652299    2.531051    2.739871    3.976684    2.408006
11715101_s_at    4.540201    5.137441    5.230140    5.703439    4.493873
11715102_x_at    3.252726    3.500768    3.883513    4.755649    3.197220
11715103_x_at    3.439989    3.527640    3.938015    3.352165    3.707648
11715104_s_at    4.226395    3.921919    4.393433    4.070046    3.989882



Platform information:
!Series_title = Integration between MCL1 gene co-expression module and the Revised International Staging System enables precise prognostication and prediction of response to proteasome inhibitor-based therapy in individual multiple myeloma
!Platform_title = [PrimeView] Affymetrix Human Gene Expression Array
!Platform_description = July 10, 2019: annotation table updated with netaffx build 36
#Target Description =
#Annotation Description =
ID	GeneChip Array	Species Scientific Name	Annotation Date	Sequence Type	Sequence Source	Transcript ID(Array Design)	Target Description	GB_ACC	GI	Representative Public ID	Archival UniGene Cluster	UniGene ID	Genome Version	Alignments	Gene Title	Gene Symbol	Chromosomal Location	Unigene Cluster Type	Ensembl	Entrez Gene	SwissProt	EC	OMIM	RefSeq Protein ID	RefSeq Transcript ID	Gene Ontology Biological Process	Gene Ontology Cellular Component	Gene Ontology Molecular Function	Pathway	InterPro	Annotation Description	Annotation Transcript


Gene annotation columns:
['ID', 'GeneChip Array', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Transcript ID(Array Design)', 'Target Description', 'GB_ACC', 'GI', 'Representative Public ID', 'Archival UniGene Cluster', 'UniGene ID', 'Genome Version', 'Alignments', 'Gene Title', 'Gene Symbol', 'Chromosomal Location', 'Unigene Cluster Type', 'Ensembl', 'Entrez Gene', 'SwissProt', 'EC', 'OMIM', 'RefSeq Protein ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function', 'Pathway', 'InterPro', 'Annotation Description', 'Annotation Transcript Cluster', 'Transcript Assignments', 'Annotation Notes', 'SPOT_ID']

Gene annotation preview:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Ge

### Step 6: Gene Identifier Mapping

In [7]:
# Based on the previous output, we've found that:
# 1. Gene expression data uses probe IDs like '11715100_at' as row indices
# 2. The gene annotation data has these probe IDs in the 'ID' column
# 3. The gene symbols are in the 'Gene Symbol' column

# 1. Extract the mapping from the gene annotation dataframe
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')

# Print preview of the mapping
print("Gene mapping (first 5 rows):")
print(gene_mapping.head())

# 2. Apply the mapping to convert probe-level measurements to gene expression values
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print shape to understand the resulting gene expression dataset dimensions
print(f"\nGene expression data after mapping (shape): {gene_data.shape}")

# Preview the first few rows of the gene expression data
print("\nGene expression data after mapping (first 5 rows, first 5 columns):")
print(gene_data.iloc[:5, :5])

# Save the gene expression data for later use
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to: {out_gene_data_file}")


Gene mapping (first 5 rows):
              ID       Gene
0    11715100_at   HIST1H3G
1  11715101_s_at   HIST1H3G
2  11715102_x_at   HIST1H3G
3  11715103_x_at  TNFAIP8L1
4  11715104_s_at      OTOP2

Gene expression data after mapping (shape): (19963, 93)

Gene expression data after mapping (first 5 rows, first 5 columns):
         GSM5712490  GSM5712491  GSM5712492  GSM5712493  GSM5712494
Gene                                                               
A1BG       4.833014    4.952351    5.425290    5.064699    6.160388
A1CF       5.632894    5.491876    5.734211    5.515223    5.677337
A2M        5.809853    6.800766    5.038784    4.354973    3.747210
A2ML1      4.576426    4.744677    4.659546    3.889619    4.376735
A3GALT2    1.890677    2.295096    2.298677    2.079080    1.970331



Gene expression data saved to: ../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/gene_data/GSE190042.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
try:
    # Now let's normalize the gene data using the provided function
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"First few gene symbols after normalization: {list(normalized_gene_data.index[:10])}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error in gene normalization: {e}")
    # If normalization fails, use the original gene data
    normalized_gene_data = gene_data
    print("Using original gene data without normalization")

# 2. Handle the case where trait data isn't available
# Recall that in step 2, we determined trait_row = None
is_trait_available = trait_row is not None
print(f"Is trait data available for {trait}? {is_trait_available}")

if not is_trait_available:
    print(f"This dataset doesn't contain data for the trait: {trait}")
    
    # When trait data isn't available, we should mark the dataset as not usable
    # for our specific trait study, even if it contains gene expression data
    
    # Creating a mock dataframe with the correct structure for validation
    df_for_validation = pd.DataFrame(index=normalized_gene_data.columns[:5], columns=[trait])
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=False,  # Using a concrete value instead of None
        df=df_for_validation,  # Passing a valid dataframe with appropriate structure
        note=f"Dataset contains gene expression data but no information about {trait}."
    )
    
    print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
    print("Data not saved due to missing trait information.")
else:
    # This block would handle the case where trait data is available
    # Since we determined in step 2 that trait_row is None, this block won't be executed
    
    # 3. Load/create the clinical data
    try:
        # Try to load clinical data from a file
        clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
        print("Loaded clinical data:")
        print(clinical_data.head())
    except Exception as e:
        print(f"Error loading clinical data: {e}")
        # Recreate clinical features using the original clinical_df
        soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)
        background_info, clinical_df = get_background_and_clinical_data(matrix_file_path)
        
        clinical_data = geo_select_clinical_features(
            clinical_df,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        ).T  # Transpose to get samples as rows
        
    # 4. Link clinical and genetic data
    gene_data_for_linking = normalized_gene_data.T
    common_samples = set(clinical_data.index).intersection(gene_data_for_linking.index)
    
    clinical_data = clinical_data.loc[clinical_data.index.isin(common_samples)]
    gene_data_for_linking = gene_data_for_linking.loc[gene_data_for_linking.index.isin(common_samples)]
    
    linked_data = pd.concat([clinical_data, gene_data_for_linking], axis=1)
    
    # 5. Handle missing values
    linked_data = handle_missing_values(linked_data, trait)
    
    # 6. Determine whether the trait and demographic features are biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 7. Quality validation and save
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_trait_biased,
        df=linked_data,
        note="Dataset with trait and gene data."
    )
    
    # 8. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print(f"Data not saved due to quality issues.")

Gene data shape after normalization: (19758, 93)
First few gene symbols after normalization: ['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AACSP1']


Normalized gene data saved to ../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/gene_data/GSE190042.csv
Is trait data available for X-Linked_Lymphoproliferative_Syndrome? False
This dataset doesn't contain data for the trait: X-Linked_Lymphoproliferative_Syndrome
Abnormality detected in the cohort: GSE190042. Preprocessing failed.
Data quality check result: Not usable
Data not saved due to missing trait information.
